In [1]:
import h5py
import glob
from tqdm import tqdm
import numpy as np
import numpy.ma as ma
import json
import sklearn.decomposition as dcomp

In [6]:
boundary_tolerance=1.0 # cm
residual_threshold=2.0 # cm
module_iogroups={0:[1,2], 1:[3,4], 2:[5,6], 3:[7,8]}
x_outer_extent=63.930999755859375
x_inner_extent=3.069000005722046
y_upper_extent=62.076
y_lower_extent=-62.076
z_outer_extent=64.538
z_inner_extent=2.4620001
drift_distance=30.

data_tpc_xrange={8:[-1*x_outer_extent,(-1*x_outer_extent)+drift_distance], 6:[-1*x_outer_extent,(-1*x_outer_extent)+drift_distance],
            7:[(-1*x_inner_extent)-drift_distance,-1*x_inner_extent], 5:[(-1*x_inner_extent)-drift_distance,-1*x_inner_extent],
            4:[x_inner_extent,x_inner_extent+drift_distance], 2:[x_inner_extent,x_inner_extent+drift_distance],
            3:[x_outer_extent-drift_distance, x_outer_extent], 1:[x_outer_extent-drift_distance, x_outer_extent]}

mc_tpc_xrange={7:[-1*x_outer_extent,(-1*x_outer_extent)+drift_distance], 5:[-1*x_outer_extent,(-1*x_outer_extent)+drift_distance],
            8:[(-1*x_inner_extent)-drift_distance,-1*x_inner_extent], 6:[(-1*x_inner_extent)-drift_distance,-1*x_inner_extent],
            3:[x_inner_extent,x_inner_extent+drift_distance], 1:[x_inner_extent,x_inner_extent+drift_distance],
            4:[x_outer_extent-drift_distance, x_outer_extent], 2:[x_outer_extent-drift_distance, x_outer_extent]}
tpc_zrange={8:[-1*z_outer_extent,-1*z_inner_extent], 6:[z_inner_extent,z_outer_extent],
            7:[-1*z_outer_extent,-1*z_inner_extent], 5:[z_inner_extent,z_outer_extent],
            4:[-1*z_outer_extent,-1*z_inner_extent], 2:[z_inner_extent,z_outer_extent],
            3:[-1*z_outer_extent,-1*z_inner_extent], 1:[z_inner_extent,z_outer_extent]}

In [ ]:
filedir='/global/cfs/cdirs/dune/www/data/2x2/simulation/productions/MiniRun6.2_1E19_RHC/MiniRun6.2_1E19_RHC.flow/FLOW/0000000/'
run_no=[]
scan_coverage(filedir, run_no, True, True)

In [ ]:
filedir='/global/cfs/cdirs/dune/www/data/2x2/nearline/flowed_charge/REFLOW/v4/beam/july10_2024/nominal_hv/'
run_no=['0050018']
scan_coverage(filedir, run_no, False, False)

  7%|▋         | 18/263 [01:39<18:08,  4.44s/it]

In [2]:
def scan_coverage(filedir, run_no, is_mc, flag=False):
    for filename in tqdm(glob.glob(filedir+'*.hdf5')):
        if is_mc==False:
            file_run_no = filename.split("/")[-1].split("-")[1] 
            if file_run_no not in run_no: continue
            dt_string = filename.split("/")[-1].split("-")[-1].split("_CDT")[0]
        if is_mc==True:
            if flag==False: # dedicated electron lifetime sample
                file_run_no=filename.split("/")[6].split("_")[-1]
                if file_run_no not in run_no: continue
                dt_string=filename.split("/")[-1].split(".")[2]
            if flag==True: # regular MC production sample
                file_run_no='minirun6_2'
                dt_string=filename.split("/")[-1].split(".")[-3]
              
        try: f = h5py.File(filename, 'r')
        except: continue
        try: ext_full = f['charge/ext_trigs/data']
        except: continue
        try: hits_full = f['charge/calib_prompt_hits/data']
        except: continue
        
        exts_ref = f['charge/events/ref/charge/ext_trigs/ref']
        ext_region = f['charge/events/ref/charge/ext_trigs/ref_region']        
        hits_ref = f['charge/events/ref/charge/calib_prompt_hits/ref']
        hits_region = f['charge/events/ref/charge/calib_prompt_hits/ref_region']

        out = {}
        event_ids=np.unique(f['charge/events/data']['id'])
        for event_id in event_ids:

            ext_ref = exts_ref[ext_region[event_id,'start']:ext_region[event_id,'stop']]
            if ext_ref.shape[0]==0: 
                ext_type=-1
                ext_iog=-1
            else:
                ext_ref = np.sort(ext_ref[ext_ref[:,0] == event_id, 1])
                ext_trigs = ext_full[ext_ref]
                ext_type=int(ext_trigs['type'][0])
                ext_iog=int(ext_trigs['iogroup'][0])
            
            hit_ref = hits_ref[hits_region[event_id,'start']:hits_region[event_id,'stop']]
            hit_ref = np.sort(hit_ref[hit_ref[:,0] == event_id, 1])
            hits = hits_full[hit_ref]
            nh=len(hits)
            if nh<50: continue

            for tpc_id in range(1,9,1):
                tpc_hits=hits[hits['io_group']==tpc_id]
                if is_mc==False:
                    mask=ma.masked_inside(tpc_hits['x'], data_tpc_xrange[tpc_id][0], data_tpc_xrange[tpc_id][1])                 
                    x=tpc_hits['x'][mask.mask]
                    y=tpc_hits['y'][mask.mask]
                    z=tpc_hits['z'][mask.mask]
                else:             
                    x=tpc_hits['x']
                    y=tpc_hits['y']
                    z=tpc_hits['z']
                nx=len(x)
                if nx<50 or nx>2000: continue

                if is_mc==False:
                    xnan=np.isnan(x); ynan=np.isnan(y); znan=np.isnan(z)
                    if np.any(xnan) or np.any(ynan) or np.any(znan): continue

                x_miss_frac, y_miss_frac, z_miss_frac = missing_extent_coverage(tpc_id, x,y,z, is_mc)
                orientation='x'; miss_frac=x_miss_frac
                if y_miss_frac<x_miss_frac:
                    orientation='y'; miss_frac=y_miss_frac
                    if z_miss_frac<y_miss_frac: orientation='z'; miss_frac=z_miss_frac
                if z_miss_frac<x_miss_frac:
                    orientation='z'; miss_frac=z_miss_frac
                    if y_miss_frac<z_miss_frac: orientation='y'; miss_frac=y_miss_frac

                abort = tpc_dimension_extreme(tpc_id, orientation, x, y, z, is_mc)
                if abort==True: continue
                
                xyz=np.concatenate((np.expand_dims(x, axis=-1), np.expand_dims(y, axis=-1), np.expand_dims(z, axis=-1),), axis=-1)
                centroid = np.mean(xyz, axis=0)
                pca = dcomp.PCA(n_components=1).fit(xyz-centroid)
                axis = pca.components_[0] / np.linalg.norm(pca.components_[0])
                if axis[1]>0: axis=-axis
                s=np.dot((xyz-centroid), axis)
                xyz_min, xyz_max = np.amin(xyz, axis=0), np.amax(xyz, axis=0)
                r_max = np.clip(centroid + axis * np.max(s), xyz_min, xyz_max)
                r_min = np.clip(centroid + axis * np.min(s), xyz_min, xyz_max)
                track_length=np.linalg.norm(r_max - r_min) # cm
                
                theta = np.arctan2(np.linalg.norm(axis[:2]), axis[-1]) # w.r.t. z-axis 
                phi = np.arctan2(axis[1], axis[0]) # about z-axis

                res=np.abs(xyz-(centroid+np.outer(s, axis)))
                count_distant=0
                for r in res:
                    residual_distance = np.sqrt((r[0]**2) + (r[1]**2) + (r[2]**2))
                    if residual_distance>residual_threshold: count_distant+=1
                residual_fraction = count_distant / len(res)

                if orientation not in out.keys(): out[orientation]={}       
                if str(tpc_id) not in out[orientation].keys(): out[orientation][str(tpc_id)]={}
                if str(event_id) not in out[orientation][str(tpc_id)].keys(): 
                    out[orientation][str(tpc_id)][str(event_id)]=[residual_fraction, miss_frac, theta, phi, track_length, ext_type, ext_iog]
        
        if len(out.keys())!=0:
            if is_mc==True:
                with open('tgm_coverage_mc_'+file_run_no+'-'+dt_string+'.json','w') as outfile: json.dump(out, outfile, indent=4)     
            else: 
                with open('tgm_coverage_'+file_run_no+'-'+dt_string+'.json','w') as outfile: json.dump(out, outfile, indent=4)

In [3]:
def tpc_dimension_extreme(tpc_id, orientation, x, y, z, is_mc):
    abort=False
    if orientation=='x':
        min_x = min(x); max_x = max(x)
        if is_mc==False: tpc_xrange=data_tpc_xrange
        if is_mc==True: tpc_xrange=mc_tpc_xrange
        if tpc_id in [1,2,3,4]:
            if abs(max_x-tpc_xrange[tpc_id][1])>boundary_tolerance: abort=True
            if abs(min_x-tpc_xrange[tpc_id][0])>boundary_tolerance: abort=True
        if tpc_id in [5,6,7,8]:
            if abs(abs(tpc_xrange[tpc_id][1])-abs(max_x))>boundary_tolerance: abort=True
            if abs(abs(tpc_xrange[tpc_id][0])-abs(min_x))>boundary_tolerance: abort=True
    if orientation=='y':
        min_y = min(y); max_y = max(y)
        if min_y>0: abort=True
        if max_y<0: abort=True
        if y_upper_extent-max_y>boundary_tolerance: abort=True
        if abs(abs(y_lower_extent)-abs(min_y))>boundary_tolerance: abort=True
    if orientation=='z':
        min_z = min(z); max_z = max(z)
        if tpc_id in [1,2,5,6]:
            if abs(max_z-tpc_zrange[tpc_id][1])>boundary_tolerance: abort=True
            if abs(min_z-tpc_zrange[tpc_id][0])>boundary_tolerance: abort=True
        if tpc_id in [3,4,7,8]:
            if abs(abs(tpc_zrange[tpc_id][1])-abs(max_z))>boundary_tolerance: abort=True
            if abs(abs(tpc_zrange[tpc_id][0])-abs(min_z))>boundary_tolerance: abort=True
    return abort

In [4]:
def missing_extent_coverage(tpc_id, x, y, z, is_mc):
    if is_mc==True:
        x_vals, x_bin_edges = np.histogram(x, bins=np.linspace(mc_tpc_xrange[tpc_id][0], mc_tpc_xrange[tpc_id][1], int(mc_tpc_xrange[tpc_id][1]-mc_tpc_xrange[tpc_id][0])+1))
        absent_x=0
        for v in x_vals:
            if v==0: absent_x+=1
        x_missing_fraction = absent_x / int(mc_tpc_xrange[tpc_id][1]-mc_tpc_xrange[tpc_id][0])
        
    if is_mc==False:
        x_vals, x_bin_edges = np.histogram(x, bins=np.linspace(data_tpc_xrange[tpc_id][0], data_tpc_xrange[tpc_id][1], int(data_tpc_xrange[tpc_id][1]-data_tpc_xrange[tpc_id][0])+1))
        absent_x=0
        for v in x_vals:
            if v==0: absent_x+=1
        x_missing_fraction = absent_x / int(data_tpc_xrange[tpc_id][1]-data_tpc_xrange[tpc_id][0])

    y_vals, y_bin_edges = np.histogram(y, bins=np.linspace(y_lower_extent, y_upper_extent, int(y_upper_extent-y_lower_extent)+1))
    absent_y=0
    for v in y_vals:
        if v==0: absent_y+=1
    y_missing_fraction = absent_y / int(y_upper_extent-y_lower_extent)

    z_vals, z_bin_edges = np.histogram(z, bins=np.linspace(tpc_zrange[tpc_id][0], tpc_zrange[tpc_id][1], int(tpc_zrange[tpc_id][1]-tpc_zrange[tpc_id][0])+1))
    absent_z=0
    for v in z_vals:
        if v==0: absent_z+=1
    z_missing_fraction = absent_z / int(tpc_zrange[tpc_id][1]-tpc_zrange[tpc_id][0])

    return x_missing_fraction, y_missing_fraction, z_missing_fraction

In [5]:
def report_tpc_dimensions(filedir, dimension):
    for filename in tqdm(glob.glob(filedir+'*.hdf5')):
        f = h5py.File(filename, 'r')
        for iog in range(1,9,1): 
            module=int((iog-1)/2)
            tpc=(iog-2)%2
            print(f['geometry_info'].attrs['module_RO_bounds'][module][tpc][dimension])
        break